# CX 4230, Spring 2016: [18] Discrete event simulation of a gas station

Recall the introduction to queueing models and discrete event simulators from the last class: [link](https://t-square.gatech.edu/access/content/group/gtc-59b8-dc03-5a67-a5f4-88b8e4d5b69a/cx4230-sp16--17-queueing.pdf). In this notebook, you will implement it.

## Exponential random numbers

Recall that in a queueing model, it is common to assume that customer interarrival times and service times are independent and identically distributed random variables. Classically, the most commonly assumed distribution is _exponential_.

More specifically, an exponentially distributed random variable $X \sim \mathcal{E}(\lambda)$ has the probability density function,

$$
  f_X(x) = \lambda \cdot \exp\left(-\frac{x}{\lambda}\right),
$$

where $\lambda$ is the mean of the distribution.

Using Numpy, these are easy to generate using the function, `numpy.random.exponential()`: http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.random.exponential.html.

Here is a quick demo.

In [1]:
from numpy.random import exponential

X_MEAN = 10.0
X_COUNT = 5
x_values = exponential (X_MEAN, X_COUNT)

print ("X ~ Exp(%g):" % X_MEAN)
for (i, x_i) in enumerate (x_values):
    print ("  X_%d = %g" % (i, x_i))

X ~ Exp(10):
  X_0 = 1.08368
  X_1 = 7.63503
  X_2 = 0.21214
  X_3 = 18.6386
  X_4 = 1.58331


As a sanity check, let's generate a large number of values and compare the sample mean to the desired (true) mean.

In [2]:
from numpy import mean

# @Demo
N_BIG = 1000
big_mean = mean (exponential (X_MEAN, N_BIG))
print ("\nSample mean of %d values: %g" % (N_BIG, big_mean))


Sample mean of 1000 values: 10.445


## Priority queues

To maintain the future event list, you need some kind of priority queue data structure. One classical choice is to use a heap, for which there is a standard implementation in Python: [link](http://www.bogotobogo.com/python/python_PriorityQueue_heapq_Data_Structure.php)

Here's a quick demo.

In [3]:
from heapq import heappush, heappop, heapify

In [4]:
# Method 1: Convert any Python list into a heap
h1 = list (x_values)
print ("Initial values:", h1)

heapify (h1)
print ("\nHeapified:", h1)

print ("\nExtracting mins...")
for i in range (len (h1)):
    print (i, ":", heappop (h1))

Initial values: [1.0836761176519276, 7.6350326868415213, 0.21213965358639475, 18.638588289987613, 1.5833073039635692]

Heapified: [0.21213965358639475, 1.5833073039635692, 1.0836761176519276, 18.638588289987613, 7.6350326868415213]

Extracting mins...
0 : 0.212139653586
1 : 1.08367611765
2 : 1.58330730396
3 : 7.63503268684
4 : 18.63858829


In [5]:
# Method 2: Insert values into the heap one at a time

print ("Inserting...")
h2 = []
for (i, x_i) in enumerate (x_values):
    print (i, ":", x_i)
    heappush (h2, x_i)
    
print ("\nHeap:", h2)
    
print ("\nExtracting minima...")
for i in range (len (h2)):
    print (i, ":", heappop (h2))

Inserting...
0 : 1.08367611765
1 : 7.63503268684
2 : 0.212139653586
3 : 18.63858829
4 : 1.58330730396

Heap: [0.21213965358639475, 1.5833073039635692, 1.0836761176519276, 18.638588289987613, 7.6350326868415213]

Extracting minima...
0 : 0.212139653586
1 : 1.08367611765
2 : 1.58330730396
3 : 7.63503268684
4 : 18.63858829


## A generic discrete event simulation engine

We can build a simple, generic discrete event simulation engine. This engine manages the future event list, which you'll recall is a priority queue of timestamped events. It continually removes the event with the lowest timestamp and processes it.

Suppose we represent an event by a tuple, `(t, e)`, where `t` is the event's timestamp and `e` is an event handler. An event handler is simply a function. Let's suppose that this function takes two arguments, `e (t, s)`, where `t` is (again) the timestamp and `s` is the system state, encoded in an application-specific way. When `e (t, s)` executes, it may update the state `s`.

**Exercise.** Complete the following function, which implements a generic discrete event simulation engine. The future events list is `events`. The initial system state is `initial_state`; the starter code below makes a copy of this state as a variable `s`, which your simulator can modify. Additionally, you should correct the last `print()` statement so that instead of a pair of `None` values it prints the timestamp and event name (name of the event handler).

In [6]:
from copy import deepcopy

def simulate (events, initial_state):
    s = deepcopy (initial_state)
    
    print ("\nFuture event list:\n%s" % str (events))
    print ("\nt=0: %s" % str (s))
        
    while events:
        # @YOUSE: Get event and process it
        (t, e) = heappop (events)
        e (t, s)
        
        #print ("t=%d: event '%s' => '%s'" % (None, None, str (s)))
        print ("t=%d: event '%s' => '%s'" % (t, e.__name__, str (s)))

## Instantiating the simulator

For the gas station model, we asked you to assume the interarrival times, pumping times, and shopping times were exponential i.i.d. random variables. So, let's start by defining some parameters for these distributions. Let's also pre-generate some number of car arrivals.

In [7]:
# Event parameters
MEAN_INTERARRIVAL_TIME = 15.0 # minutes
MEAN_PUMPING_TIME = 5.0 # minutes
MEAN_SHOPPING_TIME = 10.0 # minutes

# Number of customers (cars)
NUM_CARS = 5

# Pre-generate some interarrival times
car_interarrival_times = exponential (MEAN_INTERARRIVAL_TIME, NUM_CARS)
print ("Interrival times (in minutes) of all cars:\n", car_interarrival_times)

Interrival times (in minutes) of all cars:
 [  1.25351011   3.78744989   3.05267482  22.83193381  21.42100338]


Recall that the state consists of the logical simulation time (`now`) and three state variables: `AtPump`, `AtStore`, and `PumpFree`. Let's create this state.

In [8]:
now = 0.0 # Current (logical) simulation time
state = {'AtPump': 0          # no. cars at pump or waiting
         , 'AtStore': 0       # no. cars at store
         , 'PumpFree': True   # True <==> pump is available
        }

Let's represent an _event_ as a tuple, `(t, e)`, where `t` is the timestamp of the event and `e` is an event handler, implemented as a Python function.

If the future event list is stored in a global priority queue called `events`, the following function will insert an event into that queue.

In [9]:
def schedule (t, e):
    """
    Schedules a new event `e` at time `t`.
    """
    global events
    print ("  ==> '%s' @ t=%g" % (e.__name__, t))
    heappush (events, (t, e))

**Exercise.** Implement an event handler to process a car arrival event. Assume that event handlers take as input the timestamp `t` of the event and the state `s` of the system at time `t`.

In [10]:
def arrives (t, s):
    """
    Processes an arrival event at time `t` for a system in state `s`.
    Schedules a pumping event if the pump is free. Returns the new
    system state.
    """
    # @YOUSE
    s['AtPump'] += 1
    if s['PumpFree']:
        s['PumpFree'] = False
        schedule (t + exponential (MEAN_PUMPING_TIME), finishes)
    return s

**Exercise.** Implement a function to process the event for a car that finishes pumping gas.

In [11]:
def finishes (t, s):
    """
    Processes a finished-pumping event at time `t` for a system in
    state `s`. Schedules a pumping event if any cars are waiting.
    Returns the new system state.
    """
    # @YOUSE
    s['AtPump'] -= 1
    s['AtStore'] += 1
    schedule (t + exponential (MEAN_SHOPPING_TIME), departs)
    if s['AtPump'] > 0:
        schedule (t + exponential (MEAN_PUMPING_TIME), finishes)
    else:
        s['PumpFree'] = True
    return s

**Exercise.** Implement a function to process the event for a car that leaves the store.

In [12]:
def departs (t, s):
    """
    Processes a departure from the station event at
    time `t` for a system in state `s`.
    """
    # @YOUSE
    s['AtStore'] -= 1
    return s

**Exercise.** Create an initial future events list by converting the raw interarrival times into arrival events and inserting them into the future events list.

In [13]:
# Hint: This function may prove useful
from numpy import cumsum

events = []  # Future event list, initially empty

# @YOUSE: Create initial events from all car arrivals
for t in cumsum (car_interarrival_times):
    schedule (t, arrives)
    
# Test code
print ("\nContents of `events[:]`:")
for (i, event) in enumerate (events):
    print ("[%d] t=%g: %s" % (i, event[0], event[1].__name__))

  ==> 'arrives' @ t=1.25351
  ==> 'arrives' @ t=5.04096
  ==> 'arrives' @ t=8.09363
  ==> 'arrives' @ t=30.9256
  ==> 'arrives' @ t=52.3466

Contents of `events[:]`:
[0] t=1.25351: arrives
[1] t=5.04096: arrives
[2] t=8.09363: arrives
[3] t=30.9256: arrives
[4] t=52.3466: arrives


In [14]:
# More test code: If everything worked, so should this simulation!
simulate (events, state)


Future event list:
[(1.2535101084918805, <function arrives at 0x1049e8840>), (5.040959995107885, <function arrives at 0x1049e8840>), (8.0936348179354294, <function arrives at 0x1049e8840>), (30.925568631470099, <function arrives at 0x1049e8840>), (52.346572008225948, <function arrives at 0x1049e8840>)]

t=0: {'PumpFree': True, 'AtStore': 0, 'AtPump': 0}
  ==> 'finishes' @ t=2.28553
t=1: event 'arrives' => '{'PumpFree': False, 'AtStore': 0, 'AtPump': 1}'
  ==> 'departs' @ t=3.99141
t=2: event 'finishes' => '{'PumpFree': True, 'AtStore': 1, 'AtPump': 0}'
t=3: event 'departs' => '{'PumpFree': True, 'AtStore': 0, 'AtPump': 0}'
  ==> 'finishes' @ t=5.94144
t=5: event 'arrives' => '{'PumpFree': False, 'AtStore': 0, 'AtPump': 1}'
  ==> 'departs' @ t=6.98998
t=5: event 'finishes' => '{'PumpFree': True, 'AtStore': 1, 'AtPump': 0}'
t=6: event 'departs' => '{'PumpFree': True, 'AtStore': 0, 'AtPump': 0}'
  ==> 'finishes' @ t=10.5891
t=8: event 'arrives' => '{'PumpFree': False, 'AtStore': 0, 'AtPu